## Day 21: Fractal Art

You find a program trying to generate some art. It uses a strange process that involves repeatedly enhancing the detail of an image through a set of rules.


### Part One

The image consists of a two-dimensional square grid of pixels that are either on (#) or off (.). The program always begins with this pattern:

    .#.
    ..#
    ###

Because the pattern is both 3 pixels wide and 3 pixels tall, it is said to have a size of 3.

Then, the program repeats the following process:

 - If the size is evenly divisible by 2, break the pixels up into 2x2 squares, and convert each 2x2 square into a 3x3 square by following the corresponding enhancement rule.
 
 - Otherwise, the size is evenly divisible by 3; break the pixels up into 3x3 squares, and convert each 3x3 square into a 4x4 square by following the corresponding enhancement rule.

Because each square of pixels is replaced by a larger one, the image gains pixels and so its size increases.



In [123]:
# usual imports

import math
import numpy as np
from copy import copy

> let's define a cell, and a constructor based on a pattern.

In [124]:
class Cell:
    
    def __init__(self, dimension=2, x=0, y=0):
        assert dimension in (2, 3, 4)
        self.dimension = dimension
        size = (dimension, dimension)
        self.kernel = np.zeros(size, dtype=int)
        self.x = x
        self.y = y
        
    @property
    def pattern(self):
        buff = []
        for row in range(self.dimension):
            subuff = [
                '#' if self.kernel[row][col] == 1 else '.'
                for col in range(self.dimension)
                ]
            buff.append(''.join(subuff))
        return '/'.join(buff)        
        
    @property
    def signature(self):
        return ''.join([
            '#' if self.kernel[row][col] == 1 else '.'
            for row in range(self.dimension) 
            for col in range(self.dimension)
        ])
        
    @property
    def sum(self):
        return np.sum(self.kernel)
        
    def __str__(self):
        buff = []
        for row in range(self.dimension):
            for col in range(self.dimension):
                if self.kernel[row][col] == 1:
                    buff.append('#')
                else:
                    buff.append('.')
            buff.append('\n')
        buff = buff[0:-1]  # remove last new line
        return ''.join(buff)
    
    def __repr__(self):
        return self.signature
    
    def get(self, x, y):
        return self.kernel[y][x]
    

In [125]:
c = Cell()
assert c.x == 0
assert c.y == 0
assert c.kernel[0][0] == 0
assert c.kernel[0][1] == 0
assert c.kernel[1][0] == 0
assert c.kernel[1][1] == 0
assert c.pattern == '../..'
assert c.signature == '....'
assert c.sum == 0
print('ok')

ok


In [126]:
def new_cell(pattern='.#./..#/###', x=0, y=0):
    l = len(pattern)
    assert l in (5, 11, 19)  # include separators
    dimension = 2 if l == 5 else 3 if l == 11 else 4
    cell = Cell(dimension, x, y)
    cell.kernel = np.array([
        [1 if _ == '#' else 0 for _ in s]
        for s in pattern.split('/')
        ])
    return cell
    
# tests
        
c2 = new_cell('.#/..')
assert c2.dimension == 2
assert all([
    c2.get(0,0) == 0,
    c2.get(1,0) == 1,
    c2.get(0,1) == 0,
    c2.get(1,1) == 0,
])
assert str(c2) == '.#\n..'
assert c2.pattern == '.#/..'
assert c2.signature == '.#..'
assert c2.sum == 1

c3 = new_cell('.#./..#/###')
assert str(c3) == '.#.\n..#\n###'
assert c3.dimension == 3
assert c3.signature == '.#...####'
assert c3.sum == 5

c4 = new_cell('#.../.##./...#/#.#.')
assert c4.dimension == 4
assert str(c4) == '#...\n.##.\n...#\n#.#.'
assert c4.signature == '#....##....##.#.'
assert c4.sum == 6
print('ok')

ok


The artist's book of enhancement rules is nearby (your puzzle input); however, it seems to be missing rules. The artist explains that sometimes, one must rotate or flip the input pattern to find a match. (Never rotate or flip the output pattern, though.) Each pattern is written concisely: rows are listed as single units, ordered top-down, and separated by slashes. For example, the following rules correspond to the adjacent patterns:

    ../.#  =  ..
              .#

                    .#.
    .#./..#/###  =  ..#
                    ###

                            #..#
    #..#/..../#..#/.##.  =  ....
                            #..#
                            .##.

When searching for a rule to use, rotate and flip the pattern as necessary. For example, all of the following patterns match the same rule:

    .#.   .#.   #..   ###
    ..#   #..   #.#   ..#
    ###   ###   ##.   .#.


> Rules must be tested again rotations and vertical/horizontal flips. Let's write some
> helper functions

In [127]:
def rot90(s):
    l = list(s)
    if len(l) == 4:
        l0, l1, l2 ,l3 = l
        return ''.join([l2, l0, l3, l1])
    elif len(l) == 9:
        l0, l1, l2 ,l3, l4, l5, l6, l7, l8 = l
        return ''.join([l6, l3, l0, l7, l4, l1, l8, l5, l2])
        
assert rot90('012345678') == '630741852'
assert rot90('0123') == '2031'
assert rot90('.#...####') == '#..#.###.'
print('ok')

ok


In [104]:
def rot180(s):
    l = list(s)
    if len(l) == 4:
        l0, l1, l2 ,l3 = l
        return ''.join([l3, l2, l1, l0])
    elif len(l) == 9:
        l0, l1, l2 ,l3, l4, l5, l6, l7, l8 = l
        return ''.join([l8, l7, l6, l5, l4, l3, l2, l1, l0])
        
assert rot180('012345678') == '876543210'
assert rot180('0123') == '3210'
assert rot180('.#...####') == '####...#.'
print('ok')

ok


In [105]:
def rot270(s):
    l = list(s)
    if len(l) == 4:
        l0, l1, l2 ,l3 = l
        return ''.join([l1, l3, l0, l2])
    elif len(l) == 9:
        l0, l1, l2 ,l3, l4, l5, l6, l7, l8 = l
        return ''.join([l2, l5, l8, l1, l4, l7, l0, l3, l6])
        
assert rot270('012345678') == '258147036'
assert rot270('0123') == '1302'
assert rot270('.#...####') == '.###.#..#'
print('ok')

ok


In [106]:
def hflip(s):
    l = list(s)
    if len(l) == 4:
        l0, l1, l2 ,l3 = l
        return ''.join([l2, l3, l0, l1])
    elif len(l) == 9:
        l0, l1, l2 ,l3, l4, l5, l6, l7, l8 = l
        return ''.join([l6, l7, l8, l3, l4, l5, l0, l1, l2])
        
assert hflip('012345678') == '678345012'
assert hflip('0123') == '2301'
assert hflip('.#...####') == '###..#.#.'
print('ok')

ok


In [107]:
def vflip(s):
    l = list(s)
    if len(l) == 4:
        l0, l1, l2 ,l3 = l
        return ''.join([l1, l0, l3, l2])
    elif len(l) == 9:
        l0, l1, l2 ,l3, l4, l5, l6, l7, l8 = l
        return ''.join([l2, l1, l0, l5, l4, l3, l8, l7, l6])
        
assert vflip('012345678') == '210543876'
assert vflip('0123') == '1032'
assert vflip('.#...####') == '.#.#..###'
print('ok')

ok


> Now we can write uor Rule Class:

In [108]:
class Rule:
    
    def __init__(self, pattern, production):
        assert len(pattern), len(production) in [(5, 11), (11, 19)]  # include separators
        self.pattern = pattern
        self.dimension = 2 if len(self.pattern) == 5 else 3
        self.production = production
        
    def __str__(self):
        return '{} => {}'.format(self.pattern, self.production)
        
    def __repr__(self):
        return 'Rule("{}", "{}")'.format(self.pattern, self.production)
        
    def __eq__(self, o):
        return self.pattern, self.production == o.pattern, o.production
        
    @property
    def signature(self):
        return self.pattern.replace('/', '')
    
    def match(self, cell, tron=False):
        if self.dimension != cell.dimension:
            return False
        source = self.signature
        target = cell.signature
        if tron:
            print('- rot90', rot90(source), target, rot90(source) == target)
            print('- rot180', rot180(source), target, rot180(source) == target)
            print('- rot270', rot270(source), target, rot270(source) == target)
            print('- hflip', hflip(source), target, hflip(source) == target)
            print('- vflip', vflip(source), target, vflip(source) == target)
            # Compose vflip and rots
            print('- vflip ∘ rot90:', vflip(rot90(source)), target, vflip(rot90(source)) == target)
            print('- vflip ∘ rot180:', vflip(rot180(source)), target, vflip(rot180(source)) == target)
            print('- vflip ∘ rot270:', vflip(rot270(source)), target, vflip(rot270(source)) == target)
            # Compose hflip and rots
            print('- hflip ∘ rot90:', hflip(rot90(source)), target, hflip(rot90(source)) == target)
            print('- hflip ∘ rot180:', hflip(rot180(source)), target, hflip(rot180(source)) == target)
            print('- hflip ∘ rot270:', hflip(rot270(source)), target, hflip(rot270(source)) == target)
            
        return any([
            source == target,
            rot90(source) == target,
            rot180(source) == target,
            rot270(source) == target,
            hflip(source) == target,
            vflip(source) == target,
            vflip(rot90(source)) == target,
            vflip(rot180(source)) == target,
            vflip(rot270(source)) == target,
            hflip(rot90(source)) == target,
            hflip(rot180(source)) == target,
            hflip(rot270(source)) == target, 
        ])
    
    def apply(self, cell):
        return new_cell(self.production)
        

In [109]:
r3 = Rule('#./.#', '##./##./.##')
assert r3.match(new_cell('.#/#.')) == True
r4 = Rule('##/..', '##./##./.##')
assert r4.match(new_cell('.#/.#')) == True
print('ok')

ok


In [110]:
r1 = Rule('../..', '##./##./.##')
assert r1.match(new_cell('../..')) == True
assert r1.match(new_cell('../.#')) == False
print('ok')

ok


In [111]:
def load_rules(filename):
    rules = []
    with open(filename, 'r') as f:
        lines = [l.strip() for l in f.readlines()]
    for line in lines:
        pattern, production = line.split(' => ')
        rules.append(Rule(pattern, production))
    return rules
        
rules = load_rules('input.txt')
assert rules[0] == Rule("../..", "##./##./.##")
assert rules[1] == Rule("#./..", ".../.#./##.")
assert rules[3] == Rule("##/..", ".../.##/#.#")
...
assert rules[-3] == Rule("###/###/#.#", ".#../.#.#/.###/.#.#")
assert rules[-2] == Rule("###/#.#/###", "##../####/###./...#")
assert rules[-1] == Rule("###/###/###", "###./#..#/##../.##.")
print('ok')

ok


In [112]:
rules = load_rules('input.txt')
r = Rule('.../.#./...', '#.../.##./...#/#...')
assert r.match(new_cell('.../.#./...'))
r6 = rules[6]
assert r == rules[6]
assert r6.match(new_cell('.../.../...'), tron=True)
print('ok')

- rot90 ......... ......... True
- rot180 ......... ......... True
- rot270 ......... ......... True
- hflip ......... ......... True
- vflip ......... ......... True
- vflip ∘ rot90: ......... ......... True
- vflip ∘ rot180: ......... ......... True
- vflip ∘ rot270: ......... ......... True
- hflip ∘ rot90: ......... ......... True
- hflip ∘ rot180: ......... ......... True
- hflip ∘ rot270: ......... ......... True
ok


> And finally our workhorse class, MapofMaps. This will evolve using the rules to bigger sizes, eventualy being reduced when possible.

In [113]:
class MapOfMaps:
    
    def __init__(self, *cells):
        num_cells = len(cells)
        self.size = int(math.sqrt(num_cells))
        assert num_cells == self.size ** 2
        self.kernel = np.reshape(np.array(cells), (self.size, self.size))
        for x in range(0, self.size):
            for y in range(0, self.size):
                c = self.kernel[y][x]
                c.x = x
                c.y = y
        self.dimension = self.kernel[0][0].dimension
        assert self.dimension in (2,3,4), 'Incorrect dimension {}'.format(self.size)
        
    def get(self, x, y, tron=False):
        if tron:
            print('size: ', self.size)
            print('dimension: ', self.dimension)
            print('x:', x, 'y:', y)
        col = x // self.dimension
        row = y // self.dimension
        if tron:
            print('col:', col)
            print('row:', row)
        cell = self.kernel[row][col]
        if tron:
            print('cell:', cell)
        offset_x = x % self.dimension
        offset_y = y % self.dimension
        if tron:
            print('offset_x:', offset_x)
            print('offset_y:', offset_y)       
        return cell.get(offset_x, offset_y)
        
    def __str__(self):
        buff = []
        for y in range(self.size * self.dimension):
            if y > 0 and (y % self.dimension) == 0:
                buff.append('-'*(self.dimension*self.size) + '-'*(self.size-1)+ '\n')   
            for x in range(self.size * self.dimension):
                if x > 0 and (x % self.dimension) == 0:
                    buff.append('|')
                buff.append('#' if self.get(x, y) == 1 else '.')
            buff.append('\n')
        return ''.join(buff)
    
    def get_subcell(self, x, y):
        subcell = Cell(dimension=2, x=x//2, y=y//2)
        subcell.kernel[0][0] = self.get(x,y)
        subcell.kernel[0][1] = self.get(x+1,y)
        subcell.kernel[1][0] = self.get(x,y+1)
        subcell.kernel[1][1] = self.get(x+1,y+1)
        return subcell
        
    def check(self):
        for c in self:
            assert c.dimension == self.dimension
        
    def reduce(self, tron=False):
        if ((self.size * self.dimension) % 2 == 0) and (self.dimension > 2):
            if tron:
                print('We can reduce!')
                print('- self.size [{}] * self.dimension[{}] == {} is even'.format(
                    self.size,
                    self.dimension,
                    self.size*self.dimension,
                    ))
                print('- and self.dimmension [{}] > 2'.format(self.dimension))
            rows = []
            for y in range(0, self.size * self.dimension, 2):
                cols = []
                for x in range(0, self.size * self.dimension, 2):
                    if tron:
                        print('Try to get subcell ar x[{}], y[{}]'.format(
                            x, y))
                    cell = self.get_subcell(x, y)
                    assert cell.dimension == 2
                    cols.append(cell)
                rows.append(cols)
            self.size = self.size * self.dimension // 2
            self.dimension = 2
            self.kernel = np.array(rows)
            return True
        return False
            
    def __iter__(self):
        return (self.kernel[r][c] for r in range(self.size) for c in range(self.size))
        
    

In [114]:
mm = MapOfMaps(
    new_cell('.#./.#./.#.'),
    new_cell('###/.#./###'),
    new_cell('###/..#/###'),
    new_cell('#.#/#.#/###'),
    )

assert mm.size == 2
assert mm.dimension == 3
mm.reduce()
assert mm.size == 3
assert mm.dimension == 2
print('ok')

ok


In [115]:
mm = MapOfMaps(
    new_cell('.#./.#./.#.'),
    new_cell('###/.#./###'),
    new_cell('###/..#/###'),
    new_cell('#.#/#.#/###'),
    )

assert mm.size == 2
assert mm.dimension == 3
c00 = mm.kernel[0][0]
assert (c00.x, c00.y) == (0, 0), '{}, {} must be equal to 0,0'.format(c00.x, c00.y)
c01 = mm.kernel[0][1]
assert (c01.x, c01.y) == (1, 0), '{}, {} must be equal to 1,0'.format(c01.x, c01.y)
c10= mm.kernel[1][0]
assert (c10.x, c10.y) == (0, 1), '{}, {} must be equal to 0,0'.format(c10.x, c10.y)
c11 = mm.kernel[1][1]
assert (c11.x, c11.y) == (1, 1), '{}, {} must be equal to 0,0'.format(c11.x, c11.y)

mm.reduce()
assert mm.size == 3
assert mm.dimension == 2
print('ok')

ok


In [116]:
mm = MapOfMaps(
    new_cell('.#./.../#..'), new_cell('###/.#./###'), new_cell('###/..#/###'),
    new_cell('#.#/#.#/###'), new_cell('..#/.##/.#.'), new_cell('..#/.#./..#'),
    new_cell('.#./.#./...'), new_cell('##./#../#.#'), new_cell('##./#../#.#'),
)

assert mm.size == 3
assert mm.dimension == 3
assert mm.reduce() == False
assert mm.size == 3
assert mm.dimension == 3
print('ok')

ok


In [117]:
mm = MapOfMaps(new_cell('.#./..#/###'))
assert mm.size == 1
assert mm.dimension == 3
assert mm.reduce() == False
print(mm)
print(mm.kernel[0][0].signature)

.#.
..#
###

.#...####


In [118]:
mm = MapOfMaps(new_cell('.#./..#/###'))
r2 = Rule('.#./..#/###','..#./..#./##.#/##..')
for c in mm:
    if r2.match(c):
        mm.kernel[c.y][c.x] = r2.apply(c)
mm.dimension += 1
print(mm)
assert mm.dimension == 4
assert mm.size == 1

assert mm.reduce(tron=True) == True
assert mm.dimension == 2
assert mm.size == 2
print(mm)

..#.
..#.
##.#
##..

We can reduce!
- self.size [1] * self.dimension[4] == 4 is even
- and self.dimmension [4] > 2
Try to get subcell ar x[0], y[0]
Try to get subcell ar x[2], y[0]
Try to get subcell ar x[0], y[2]
Try to get subcell ar x[2], y[2]
..|#.
..|#.
-----
##|.#
##|..



In [ ]:
def expand(mom, rules, try_reduce=True, tron=False):
    mom.check()
    if tron:
        print(mom)
    for c in mom:
        if tron:
            print('Checking cell', c, end=' --> ')
        for rule in rules:
            if rule.match(c):
                break
        else:
            raise ValueError("con't find a rule to apply")
                
        if tron:
            print('- rule {} match cell {},{}'.format(rule, c.x, c.y))
        new_cell = rule.apply(c)
        new_cell.x = c.x
        new_cell.y = c.y
        mom.kernel[c.y][c.x] = new_cell
    print('Dimension goes from {}'.format(mm.dimension), end=' -> ')
    mom.dimension += 1
    print('to {}'.format(mm.dimension))
    if try_reduce:
        flag = mom.reduce(tron)
        if tron:
            if flag:
                print('dimension now is {}'.format(mm.dimension))
            else:
                print("can't reduce")
    return mom

# tests

Suppose the book contained the following two rules:

    ../.# => ##./#../...
    .#./..#/### => #..#/..../..../#..#

As before, the program begins with this pattern:

    .#.
    ..#
    ###

The size of the grid (3) is not divisible by 2, but it is divisible by 3. It divides evenly into a single square; the square matches the second rule, which produces:

    #..#
    ....
    ....
    #..#

The size of this enhanced grid (4) is evenly divisible by 2, so that rule is used. It divides evenly into four squares:

    #.|.#
    ..|..
    --+--
    ..|..
    #.|.#

Each of these squares matches the same rule (../.# => ##./#../...), three of which require some flipping and rotation to line up with the rule. The output for the rule is the same in all four cases:

    ##.|##.
    #..|#..
    ...|...
    ---+---
    ##.|##.
    #..|#..
    ...|...

Finally, the squares are joined into a new grid:

    ##.##.
    #..#..
    ......
    ##.##.
    #..#..
    ......

Thus, after 2 iterations, the grid contains 12 pixels that are on.

In [130]:
rules = load_rules('input_test.txt')
mm = MapOfMaps(new_cell('.#./..#/###'))

print('Estado inicial:')
print(mm)
assert mm.size == 1
assert mm.dimension == 3

print('Primera expansion')
mm = expand(mm, rules)
print(mm)
assert mm.size == 2
assert mm.dimension == 2

print('Segunda expansion')
mm = expand(mm, rules)
print(mm)
assert mm.size == 3
assert mm.dimension == 2

print('Num of active cells:', sum(c.sum for c in mm))

Estado inicial:
.#.
..#
###

Primera expansion
Dimension goes from 3 -> to 4
#.|.#
..|..
-----
..|..
#.|.#

Segunda expansion
Dimension goes from 2 -> to 3
##|.#|#.
#.|.#|..
--------
..|..|..
##|.#|#.
--------
#.|.#|..
..|..|..

Num of active cells: 12


In [131]:
# The real thing

rules = load_rules('input.txt')
mm = MapOfMaps(new_cell('.#./..#/###'))

print(mm)
assert mm.size == 1
assert mm.dimension == 3
for i in range(3):
    print('Expand phase {}'.format(i+1))
    mm = expand(mm, rules)
print('Final map')
print(mm)

print('Expand phase 4')
assert mm.size == 3
assert mm.dimension == 3
mm = expand(mm, rules)
assert mm.size == 6
assert mm.dimension == 2

print('Expand phase 5')
assert mm.size == 6
assert mm.dimension == 2
mm = expand(mm, rules)
assert mm.size == 9
assert mm.dimension == 2

def  show_map(mm):
    for y in range(mm.size*mm.dimension):
        for x in range(mm.size*mm.dimension):
            try:
                v = mm.get(x, y)
                print('□' if v == 0 else '▣', end='')
            except IndexError:
                print('☠', end='')
        print()
        
show_map(mm)

.#.
..#
###

Expand phase 1
Dimension goes from 3 -> to 4
Expand phase 2
Dimension goes from 2 -> to 3
Expand phase 3
Dimension goes from 2 -> to 3
Final map
...|.##|...
.#.|#.#|.#.
##.|#..|##.
-----------
...|.##|...
.#.|#.#|.#.
##.|#..|##.
-----------
.##|...|##.
#.#|.##|##.
#..|#.#|.##

Expand phase 4
Dimension goes from 3 -> to 4
Expand phase 5
Dimension goes from 2 -> to 3
□▣▣▣▣□□□□□□□□▣▣▣▣□
▣□▣▣□□□▣□□▣▣▣□▣▣□□
▣□□▣□□▣▣□▣□▣▣□□▣□□
□□□□□□□▣▣□□□□□□□□□
□▣□□▣▣▣□▣□▣□□▣□□▣▣
▣▣□▣□▣▣□□▣▣□▣▣□▣□▣
□▣▣▣▣□□□□□□□□▣▣▣▣□
▣□▣▣□□□▣□□▣▣▣□▣▣□□
▣□□▣□□▣▣□▣□▣▣□□▣□□
□□□□□□□▣▣□□□□□□□□□
□▣□□▣▣▣□▣□▣□□▣□□▣▣
▣▣□▣□▣▣□□▣▣□▣▣□▣□▣
□□□□□□□□▣□□□▣▣□□□▣
□▣□□▣▣□▣□□▣▣▣□□□▣□
▣▣□▣□▣□▣▣▣□▣▣□□□▣▣
□▣▣□□□▣▣□□□▣▣▣□□▣▣
▣□▣□▣□▣▣□□▣□▣□□▣□▣
▣□□▣▣□□▣▣□▣▣▣□□▣□□


**How many pixels stay on after 5 iterations?**

In [132]:
print('Part one:', sum(c.sum for c in mm))

Part one: 144


### Part Two

**How many pixels stay on after 18 iterations?**

In [133]:
rules = load_rules('input.txt')
mm = MapOfMaps(new_cell('.#./..#/###'))
for _ in range(18):
    mm = expand(mm, rules)
print('Part two:', sum(c.sum for c in mm))

Dimension goes from 3 -> to 4
Dimension goes from 2 -> to 3
Dimension goes from 2 -> to 3
Dimension goes from 3 -> to 4
Dimension goes from 2 -> to 3
Dimension goes from 2 -> to 3
Dimension goes from 3 -> to 4
Dimension goes from 2 -> to 3
Dimension goes from 2 -> to 3
Dimension goes from 3 -> to 4
Dimension goes from 2 -> to 3
Dimension goes from 2 -> to 3
Dimension goes from 3 -> to 4
Dimension goes from 2 -> to 3
Dimension goes from 2 -> to 3
Dimension goes from 3 -> to 4
Dimension goes from 2 -> to 3
Dimension goes from 2 -> to 3
Part two: 2169301
